In [1]:
import numpy as np
import math
import Aes_encryption as encryp


Nb=Nk=np.uint8(4)                         # here Nb is the number of columns(32 bit words) in the state arrary and Nk is the number of columns
                                # (32 bit words) in key array, Nk could be 4,6,8 but for this case it is 4

Nr=np.uint8(10)                           # Nr is the number of rounds which is a funciton of Nk and Nb (which is fixed). for this standard Nr = 10
inverse_mixcolumn_const=np.array([14,11,13,9,9,14,11,13,13,9,14,11,11,13,9,14],dtype=np.uint8)
inverse_mixcolumn_const=np.reshape(inverse_mixcolumn_const,(4,4))


# RSbox creation for the Gf(2^8)

r_s_box = np.array([0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3,
                    0x9e, 0x81, 0xf3, 0xd7, 0xfb , 0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f,
                    0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb , 0x54,
                    0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b,
                    0x42, 0xfa, 0xc3, 0x4e , 0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24,
                    0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25 , 0x72, 0xf8,
                    0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d,
                    0x65, 0xb6, 0x92 , 0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda,
                    0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84 , 0x90, 0xd8, 0xab,
                    0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3,
                    0x45, 0x06 , 0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1,
                    0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b , 0x3a, 0x91, 0x11, 0x41,
                    0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6,
                    0x73 , 0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9,
                    0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e , 0x47, 0xf1, 0x1a, 0x71, 0x1d,
                    0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b ,
                    0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0,
                    0xfe, 0x78, 0xcd, 0x5a, 0xf4 , 0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07,
                    0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f , 0x60,
                    0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f,
                    0x93, 0xc9, 0x9c, 0xef , 0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5,
                    0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61 , 0x17, 0x2b,
                    0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55,
                    0x21, 0x0c, 0x7d],dtype=np.uint8)
                                             # this representation is in hexadecimal format so while printing it, we wil get corresponding integers

In [2]:
# most of the lambdas and short methods are here

character_conversion=np.vectorize(chr)       # this is to conver a numpy uint8 array to its unicode containing numpy array

inverse_byte_substitution= np.vectorize(lambda index: r_s_box[index])
                                             # a copy of state_array or array for which subtitution is required should be passed in it.

In [15]:
# all the definitions/methods are here

def encrypted_text_read(filename,debug_mode=False,debug_list=None):
    
    if not debug_mode:
        state_array= np.loadtxt(filename,dtype=str,delimiter=" ",encoding="utf-8")[:-1].astype(np.uint8)                  
    else:        
        state_array= np.array(debug_list, dtype= np.uint8)
    
                                                                        # store them into utf-8 integer encoding in a variable of 8-bit integer.
                                                                        # here is we have no need of padding zeroes at the end because the
                                                                        # filtered input from loadtxt line of code is already in multiple of 16.
    state_array= np.array([np.reshape(i,(4,4)) for i in np.split(state_array,len(state_array)/16)])                        
    return state_array 

In [16]:
def input_text_key(filename, debug_mode=False, debug_list=None):
    
    if not debug_mode:
        with open(filename,'r',encoding='utf-8') as f:
            result=list(map(ord,f.read()))
                                                                        # note that 'map' can work only once, so store it result in a varaiable
    else:
        result= debug_list
    state_array= np.array(result, dtype=np.uint8)                  
                                                                        # store them into utf-8 integer encoding in a variable of 8-bit integer.

    length= len(state_array)
    if length >= 16:
        state_array= state_array[:16]
                                                                # here neglecting the rest of the file txt if more than 16 characters
    else:
        padding= length%16
        state_array= np.append(state_array,np.zeros(16-padding).astype(np.uint8))
                                                                # here adding the rest of characters as 0 if have less than 16 
                                                                # character in the input text key file.              
    
    state_array= np.array([np.reshape(state_array,(4,4)).transpose()])                        
    return state_array                                             
                                                                # reshaping the array as a 4x4 matrix and using list to encapsulate it 
                                                                # into another np.array to have the structure similar to the 
                                                                # state_array

In [17]:
def invshiftrows(element):
    temp = np.zeros(element.shape,dtype=np.uint8)
    for r in range(4):
        for j in range(Nb):
            temp[r,(encryp.shift(r,Nb)+j)%Nb]= element[r,j]
    element[:,:]=np.copy(temp)
                    #  don't panic about assignment to value of 'element' reference as it is working absloutely fine, moreover remember that
                    # list are immutable and that they are called by reference in functions and not called by value

In [18]:
#inverse_multiplication_for_matrix(inverse_mixcolumn_const,cipher_input) is the prototype for this
def inverse_multiplication_for_matrix(X,Y):
    # iterate through rows of X
    rough= np.zeros((len(X),len(Y[0])),dtype= np.uint8)
    for i in range(len(X)):
        # iterate through columns of Y
        for j in range(len(Y[0])):
            # iterate through rows of Y
            for k in range(len(Y)):
                if X[i,k]==14:
                    a=b=c=Y[k,j]
                    a= encryp.lefShift_xor(a);a= encryp.lefShift_xor(a);a= encryp.lefShift_xor(a)
                    b= encryp.lefShift_xor(b);b= encryp.lefShift_xor(b)
                    c= encryp.lefShift_xor(c)
                    rough[i,j] = rough[i,j]^(a^b^c)
                    
                if X[i,k]== 9:
                    a=Y[k,j]
                    a= encryp.lefShift_xor(a);a= encryp.lefShift_xor(a);a= encryp.lefShift_xor(a)
                    rough[i,j] = rough[i,j]^(a^Y[k,j])
                    
                if X[i,k]== 13:
                    a=b=Y[k,j]
                    a= encryp.lefShift_xor(a);a= encryp.lefShift_xor(a);a= encryp.lefShift_xor(a)
                    b= encryp.lefShift_xor(b);b= encryp.lefShift_xor(b)
                    rough[i,j] = rough[i,j]^(a^b^Y[k,j])
                
                if X[i,k]==11:
                    a=b=Y[k,j]
                    a= encryp.lefShift_xor(a);a= encryp.lefShift_xor(a);a= encryp.lefShift_xor(a)
                    b= encryp.lefShift_xor(b)
                    rough[i,j] = rough[i,j]^(a^b^Y[k,j])
                                        
    Y[:,:]=rough[:,:]

In [19]:
# the final method to store the output of the decrypted array into a passed filename
def storeOutput(filename,state_array_out):
    char_written_length= 0
    with open(filename, 'w', encoding= 'utf-8') as f:
        for i in state_array_out:
            rowwritten= ''.join(map(chr, np.ravel(i,order='F')))
            char_written_length=char_written_length+len(rowwritten)
            f.write(rowwritten)
    
    return char_written_length

In [20]:
# only the block of cipher should be passed that need to be decrypted and not the whole cipher,containing all the blocks of the cipher

def final_decryption(cipher_input,expanded_key):
    #breakpoint()
    round_number= 10
    encryp.add_round_key(cipher_input,expanded_key[round_number,:,:])
                                 # this has been done because 0 round_key should be added before any processing of the input cipher
    for round_number in range(9,0,-1):
        invshiftrows(cipher_input)
        cipher_input= inverse_byte_substitution(cipher_input)
        encryp.add_round_key(cipher_input,expanded_key[round_number,:,:])
        inverse_multiplication_for_matrix(inverse_mixcolumn_const,cipher_input)
                                 # one more thing that has been done here is that passing only the necessary part of the 
                                 # expanded_key with no passing of round number is needed then(also round_number variable
                                 # should also be used in passing the expanded_key as follows expanded_key[round_number,:,:])

                                 # above is the process for the round 9 to round 1

    round_number= 0
    invshiftrows(cipher_input)
    cipher_input= inverse_byte_substitution(cipher_input)
    encryp.add_round_key(cipher_input,expanded_key[round_number,:,:])
                                # above is the process for the round 10 only

    return cipher_input

In [21]:
# code for creation of state array and performing encryption on all blocks of the cipher created
def creation_everything(debug_mode=False, debug_list1=None, debug_list2=None):
    filename= input("enter the name of the file with path that need to be decrypted")
    encryption_key= input("enter the name of the key file with path that need to decrypt the file, max length of file is 16 characters")
    OutFileName= input("enter the name of the output file with path that is used to store the decrypted output")
    state_array= encrypted_text_read(filename, debug_mode, debug_list1)
    original_key= input_text_key(encryption_key, debug_mode, debug_list2)
    expanded_key= encryp.keyexpansion(np.copy(original_key))
    state_array_out= np.zeros(state_array.shape,dtype=np.uint8)

                                        
    for index,block in enumerate(state_array):
        state_array_out[index]= final_decryption(np.copy(block),expanded_key)
    
    if not debug_mode:
        total_char_wrote= storeOutput(OutFileName,np.copy(state_array_out))
        print(state_array_out)  # for debugging purouse only
        print(encryp.input_text(OutFileName))  # for debugging purouse only
        return np.array_equal(encryp.input_text(OutFileName),state_array_out), total_char_wrote
    else:
        hex_conversion=np.vectorize(hex)
        print(hex_conversion(state_array_out))
    

In [22]:
string="69 c4 e0 d8 6a 7b 04 30 d8 cd b7 80 70 b4 c5 5a"
list1= list(map(lambda x: int(x,16),string.split()))
string= "00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f"
list2= list(map(lambda x: int(x,16),string.split()))
print(list1,"\n",list2)

[105, 196, 224, 216, 106, 123, 4, 48, 216, 205, 183, 128, 112, 180, 197, 90] 
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [23]:
string="60 a7 41 ba 71 1c 4c 6b 4a 76 3d 59 ed 1c 62 b5"
list1= list(map(lambda x: int(x,16),string.split()))
string= "00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f"
list2= [104, 101, 108, 108, 111, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48]
print(list1,"\n",list2)

[96, 167, 65, 186, 113, 28, 76, 107, 74, 118, 61, 89, 237, 28, 98, 181] 
 [104, 101, 108, 108, 111, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48]


In [14]:
creation_everything(True,list1,list2)


enter the name of the file with path that need to be decrypted 
enter the name of the key file with path that need to decrypt the file, max length of file is 16 characters 
enter the name of the output file with path that is used to store the decrypted output 


[[['0x31' '0x54' '0x69' '0x65']
  ['0x32' '0x68' '0x73' '0x0']
  ['0x33' '0x69' '0x74' '0x0']
  ['0x34' '0x73' '0x68' '0x0']]]


In [45]:
chr(101)

'e'

In [25]:
creation_everything()
# this output shows that the problem is somewhere about reading and writing the output

enter the name of the file with path that need to be decrypted output_demo.txt
enter the name of the key file with path that need to decrypt the file, max length of file is 16 characters Text_File.txt
enter the name of the output file with path that is used to store the decrypted output decrypt.txt


[[[ 49  53 105 115]
  [ 50  32 115  32]
  [ 51 116  32 116]
  [ 52 104 105 104]]

 [[101  97  99 101]
  [ 32  99 101 115]
  [112 116  32 116]
  [114 105 116  32]]

 [[107  49  32  32]
  [101  50  97  99]
  [121  51 110  97]
  [ 32  52 100 110]]

 [[ 32 110 115  32]
  [ 98  97  32  97]
  [101 109  97  32]
  [ 32 101 115 116]]

 [[114  32  32 111]
  [105 107  97  32]
  [ 97 101 108 105]
  [108 121 115  32]]

 [[119 100 121  97]
  [111  32  32 116]
  [117 115 116  32]
  [108  97 104 105]]

 [[ 32 101  32 116]
  [104  32 119 101]
  [ 97 116 114  32]
  [118 111 105 115]]

 [[111 104  32 105]
  [109 105 105 116]
  [101 110 110  46]
  [116 103  32  10]]]
[[[ 49  53 105 115]
  [ 50  32 115  32]
  [ 51 116  32 116]
  [ 52 104 105 104]]

 [[101  97  99 101]
  [ 32  99 101 115]
  [112 116  32 116]
  [114 105 116  32]]

 [[107  49  32  32]
  [101  50  97  99]
  [121  51 110  97]
  [ 32  52 100 110]]

 [[ 32 110 115  32]
  [ 98  97  32  97]
  [101 109  97  32]
  [ 32 101 115 116]]

 [[114  32  32 1

(True, 128)

In [38]:
print("hey there \r i am using it")

 i am using it


In [35]:
chr(13)

'\r'

In [36]:
ord(chr(10))

10

In [39]:
chr(10)

'\n'